In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

# Extracting features of 2022 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])


bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN
1,JANUARY,7.0,The Legend of La Llorona,Saban Films,Patricia Harris Seeley (director/screenplay); ...,[3],NaN
2,JANUARY,7.0,The Commando,Saban Films,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN
3,JANUARY,14.0,Scream,Paramount Pictures / Spyglass Media Group,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[5],NaN
4,JANUARY,14.0,Hotel Transylvania: Transformania,Amazon Studios / Columbia Pictures / Sony Pict...,"Jennifer Kluska, Derek Drymon (directors); Amo...",[6],NaN
...,...,...,...,...,...,...,...
160,DECEMBER,16.0,Shazam! Fury of the Gods,Warner Bros. Pictures / New Line Cinema / DC F...,"David F. Sandberg (director); Henry Gayden, Ch...",NaN,[143]
161,DECEMBER,21.0,Untitled Mario film,Universal Pictures / Illumination / Nintendo,"Aaron Horvath, Michael Jelenic (directors); Ma...",NaN,[144]
162,DECEMBER,21.0,I Wanna Dance with Somebody,TriStar Pictures / Black Label Media,Kasi Lemmons (director); Anthony McCarten (scr...,NaN,[51]
163,DECEMBER,25.0,Babylon,Paramount Pictures,Damien Chazelle (director/screenplay); Brad Pi...,NaN,[145]


In [10]:
df_2022 = df[['Title','Cast and crew']]

In [11]:
df_2022

,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo..."
...,...,...
160,Shazam! Fury of the Gods,"David F. Sandberg (director); Henry Gayden, Ch..."
161,Untitled Mario film,"Aaron Horvath, Michael Jelenic (directors); Ma..."
162,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...
163,Babylon,Damien Chazelle (director/screenplay); Brad Pi...


In [12]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '1807cdd6e32d6a41911062f2c73e24a9'

In [13]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [14]:
df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))

E:\TEMP\ipykernel_7544\2369738557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))


In [15]:
df_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Horror Thriller
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo...",Animation Family Fantasy Comedy Adventure
...,...,...,...
160,Shazam! Fury of the Gods,"David F. Sandberg (director); Henry Gayden, Ch...",Comedy Action Fantasy
161,Untitled Mario film,"Aaron Horvath, Michael Jelenic (directors); Ma...",NaN
162,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama
163,Babylon,Damien Chazelle (director/screenplay); Brad Pi...,Action Adventure Science Fiction Thriller


In [16]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [17]:
df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))

E:\TEMP\ipykernel_7544\3796821595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))


In [18]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [19]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))

E:\TEMP\ipykernel_7544\2929607607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [20]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [21]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))

E:\TEMP\ipykernel_7544\2384233428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [22]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [23]:
df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [24]:
df_2022

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Horror Thriller,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo...",Animation Family Fantasy Comedy Adventure,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn
...,...,...,...,...,...,...,...
160,Shazam! Fury of the Gods,"David F. Sandberg (director); Henry Gayden, Ch...",Comedy Action Fantasy,David F. Sandberg,Zachary Levi,Jack Dylan Grazer,Rachel Zegler
161,Untitled Mario film,"Aaron Horvath, Michael Jelenic (directors); Ma...",NaN,"Aaron Horvath, Michael Jelenic",Chris Pratt,Anya Taylor-Joy,Charlie Day
162,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci
163,Babylon,Damien Chazelle (director/screenplay); Brad Pi...,Action Adventure Science Fiction Thriller,Damien Chazelle,Brad Pitt,Margot Robbie,Tobey Maguire


In [25]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [26]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [27]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Horror Thriller,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando
3,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream
4,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Family Fantasy Comedy Adventure,Hotel Transylvania: Transformania
...,...,...,...,...,...,...
160,David F. Sandberg,Zachary Levi,Jack Dylan Grazer,Rachel Zegler,Comedy Action Fantasy,Shazam! Fury of the Gods
161,"Aaron Horvath, Michael Jelenic",Chris Pratt,Anya Taylor-Joy,Charlie Day,NaN,Untitled Mario film
162,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama,I Wanna Dance with Somebody
163,Damien Chazelle,Brad Pitt,Margot Robbie,Tobey Maguire,Action Adventure Science Fiction Thriller,Babylon


In [28]:
new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']

In [29]:
new_df22.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      6
actor_3_name     11
genres            2
movie_title       1
comb             12
dtype: int64

In [30]:
new_df22 = new_df22.dropna(how='any')

In [31]:
new_df22.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [32]:
new_df22['movie_title'] = new_df22['movie_title'].str.lower()

E:\TEMP\ipykernel_7544\2732328091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22['movie_title'] = new_df22['movie_title'].str.lower()


In [33]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,the 355,Jessica Chastain Lupita Nyong'o Penélope Cruz ...
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Horror Thriller,the legend of la llorona,Autumn Reeser Antonio Cupo Danny Trejo Patrici...
3,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
4,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Family Fantasy Comedy Adventure,hotel transylvania: transformania,Andy Samberg Selena Gomez Kathryn Hahn Jennife...
5,Luis Prieto,Cameron Monaghan,Frank Grillo,Lilly Krug,Thriller,shattered,Cameron Monaghan Frank Grillo Lilly Krug Luis ...
...,...,...,...,...,...,...,...
158,Tommy Wirkola,David Harbour,John Leguizamo,Beverly D'Angelo,Action Thriller,violent night,David Harbour John Leguizamo Beverly D'Angelo ...
159,James Cameron,Sam Worthington,Zoe Saldana,Stephen Lang,Action Adventure Science Fiction Fantasy,avatar 2,Sam Worthington Zoe Saldana Stephen Lang James...
160,David F. Sandberg,Zachary Levi,Jack Dylan Grazer,Rachel Zegler,Comedy Action Fantasy,shazam! fury of the gods,Zachary Levi Jack Dylan Grazer Rachel Zegler D...
162,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama,i wanna dance with somebody,Naomi Ackie Ashton Sanders Stanley Tucci Kasi ...


In [34]:
old_df = pd.read_csv('final_data.csv')

In [35]:
final_df = pd.concat([old_df, new_df22], ignore_index=True)

In [36]:
final_df.to_csv('./final_data.csv',index=False)

In [37]:
final_df.to_csv('main_data.csv',index=False)